# Gradio Example

In [0]:
import gradio as gr
import chromadb
import pandas as pd

DATA_DIR = "src/data"
RESEARCH_PAPER_DB = f"{DATA_DIR}/papers_titles_&_abstracts_1470.csv"

papers_df = pd.read_csv(RESEARCH_PAPER_DB)
# collection = chroma_client.create_collection("example_database")
chroma_client = chromadb.PersistentClient(path="src/vector_database/chroma_data/")
collection = chroma_client.get_collection("example_database")
chroma_client.list_collections()

In [0]:
# To create an example collection
# document_list = [row["PAPER_TITLE"] for idx, row in papers_df.iterrows() if idx < 20]
# document_ids = [row["PAPER_LINK"] for idx, row in papers_df.iterrows() if idx < 20]
# collection.add(
#     documents=document_list,
#     ids=document_ids,
# )

In [0]:
test_result = collection.query(
    query_texts=["give me research on heart conditions"],
    n_results=5,
)

In [0]:
# We can embed more information here, such as include the abstract within the metadata to
# then be pulled for summarisation
test_result

In [0]:
def query(query_text: str):
    results = collection.query(
        query_texts=[query_text],
        n_results=5,
    )
    similar_docs = results["documents"][0]

    return pd.DataFrame({"Relevant Documents": similar_docs}), "summarised_text"

In [0]:
with gr.Blocks(theme=gr.themes.base.Base(), title="Ask us anything") as demo:
    gr.Markdown(
        """
        # Retreive relevant research papers to your query
        """
    )
    textbox = gr.Textbox(label="Please enter your question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output_summary = gr.Textbox(label="Summary:")
        output = gr.List(type="pandas", label="Relevant documents:")

    button.click(query, textbox, outputs=[output, output_summary])

In [0]:
demo.launch()